# Explore here

In [155]:
# Your code here
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

movies_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

In [144]:
movies_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [145]:
credits_data.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [146]:
conn = sqlite3.connect("movies_database.db")
movies_data.to_sql("movies", conn, if_exists="replace", index=True)
credits_data.to_sql("credits", conn, if_exists="replace", index=True)

4803

In [147]:
cursor = conn.cursor()
query = """
SELECT * FROM movies
JOIN credits ON movies.id = credits.movie_id
"""
cursor.execute(query)

columns = [description[0] for description in cursor.description]
results = cursor.fetchall()
print(columns)

['index', 'budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'index', 'movie_id', 'title', 'cast', 'crew']


In [148]:
columns_to_preserved = ["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]
filtered_results = []

for row in results:
    filtered_row = {column: value for column, value in zip(columns, row) if column in columns_to_preserved}
    filtered_results.append(filtered_row)


In [149]:
df = pd.DataFrame(filtered_results)
df.sample(3)

,genres,keywords,overview,title,movie_id,cast,crew
2458,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 579, ""name"": ""american football""}, {""i...","In small-town Texas, high school football is a...",Varsity Blues,14709,"[{""cast_id"": 1, ""character"": ""Jonathan 'Mox' M...","[{""credit_id"": ""52fe46129251416c7506c24b"", ""de..."
4084,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 18, ""...","[{""id"": 6281, ""name"": ""partnership""}, {""id"": 1...",McMasters and Sand come to oil towns to get ri...,Boom Town,55604,"[{""cast_id"": 1, ""character"": ""Big John McMaste...","[{""credit_id"": ""57191cddc3a3686d3000527a"", ""de..."
642,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10752, ""n...","[{""id"": 1956, ""name"": ""world war ii""}, {""id"": ...","A chronicle of the life of Louis Zamperini, an...",Unbroken,227306,"[{""cast_id"": 11, ""character"": ""Louis Zamperini...","[{""credit_id"": ""531b62dd92514177b1004164"", ""de..."


In [150]:
def process_json_column(column, mode="all", limit=None):
    if not isinstance(column, str):
        return None
    
    try:
        json_list = json.loads(column)
    except json.JSONDecodeError:
        return None
    
    if not isinstance(json_list, list):
        return None
    
    names = [item['name'] for item in json_list if isinstance(item, dict) and "name" in item]
    director = next((item.get('director') for item in json_list if isinstance(item, dict) and "director" in item), "")
    
    if mode == "limit" and limit:
        names = names[:limit]

    return ", ".join(names) if not director else director


In [151]:
df["genres"] = df["genres"].apply(lambda x: process_json_column(x, mode="all"))
df["keywords"] = df["keywords"].apply(lambda x: process_json_column(x, mode="all"))
df["cast"] = df["cast"].apply(lambda x: process_json_column(x, mode="limit", limit=3))
df["crew"] = df["crew"].apply(lambda x: process_json_column(x, mode="all"))

df.head(3)

,genres,keywords,overview,title,movie_id,cast,crew
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"Sam Worthington, Zoe Saldana, Sigourney Weaver","Stephen E. Rivkin, Rick Carter, Christopher Bo..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"Johnny Depp, Orlando Bloom, Keira Knightley","Dariusz Wolski, Gore Verbinski, Jerry Bruckhei..."
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"Daniel Craig, Christoph Waltz, Léa Seydoux","Thomas Newman, Sam Mendes, Anna Pinnock, John ..."


In [152]:
overview_list = df["overview"].tolist()
df["overview"] = overview_list

df.head(3)

,genres,keywords,overview,title,movie_id,cast,crew
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"Sam Worthington, Zoe Saldana, Sigourney Weaver","Stephen E. Rivkin, Rick Carter, Christopher Bo..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"Johnny Depp, Orlando Bloom, Keira Knightley","Dariusz Wolski, Gore Verbinski, Jerry Bruckhei..."
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",A cryptic message from Bond’s past sends him o...,Spectre,206647,"Daniel Craig, Christoph Waltz, Léa Seydoux","Thomas Newman, Sam Mendes, Anna Pinnock, John ..."


In [153]:
df['tags'] = df[['genres', 'cast', 'crew', 'keywords', 'overview']].apply(lambda x: ' '.join(x.dropna()), axis=1)

df['tags'] = df['tags'].str.replace(r',', ' ', regex=True)

print(df['tags'].iloc[0])

df.head(2)

Action  Adventure  Fantasy  Science Fiction Sam Worthington  Zoe Saldana  Sigourney Weaver Stephen E. Rivkin  Rick Carter  Christopher Boyes  Christopher Boyes  Mali Finn  James Horner  James Cameron  James Cameron  James Cameron  James Cameron  James Cameron  Andrew Menzies  Jill Brooks  Margery Simkin  Kevin Ishioka  Dick Bernstein  Shannon Mills  Dennie Thorpe  Jana Vance  Deborah Lynn Scott  Jon Landau  Sean Haworth  Kim Sinclair  Kim Sinclair  Richard F. Mays  Laeta Kalogridis  Mayes C. Rubeo  Mauro Fiore  Scott Herbertson  Woody Schultz  Linda DeVetta  Linda DeVetta  Richard Bluck  Simon Bright  Richard Martin  Steve R. Moore  John Refoua  Karl J. Martin  Chiling Lin  Ilram Choi  Steven Quale  Carla Meyer  Nick Bassett  Jill Cormack  Andy McLaren  Terry Notary  Garrett Warren  Jonathan Rothbart  Stefan Dechant  Todd Cherniawsky  Miranda Rivers  Robert Stromberg  John Harding  Roberto De Angelis  Mike Smithson  Alain Lalanne  Lucas Salton  Janace Tashjian  Stephen Rosenbaum  Frank

,genres,keywords,overview,title,movie_id,cast,crew,tags
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,"Sam Worthington, Zoe Saldana, Sigourney Weaver","Stephen E. Rivkin, Rick Carter, Christopher Bo...",Action Adventure Fantasy Science Fiction Sa...
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,"Johnny Depp, Orlando Bloom, Keira Knightley","Dariusz Wolski, Gore Verbinski, Jerry Bruckhei...",Adventure Fantasy Action Johnny Depp Orland...


In [154]:
df.to_csv("../data/processed/clean_data.csv", index = False)

conn = sqlite3.connect("../data/movies_database.db")

movies_data.to_sql("clean_movies_data", conn, if_exists = "replace", index = False)

4803

In [156]:
vectorizer = TfidfVectorizer()
tfidf_vectors = vectorizer.fit_transform(df["tags"])

In [160]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(tfidf_vectors)

model = NearestNeighbors(n_neighbors = 6, algorithm = "auto", metric = "cosine")
model.fit(tfidf_vectors)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [167]:
def recommend(movie, similarity):
    movie_index = df[df["title"] == movie].index[0]
    
    distances = similarity[movie_index]
    
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(df.iloc[i[0]].title)

recommend("Avatar", similarity)


Aliens
Jurassic World
Guardians of the Galaxy
John Carter
A.I. Artificial Intelligence


In [168]:
from pickle import dump

dump(model, open("../models/knn_neighbors-6_algorithm-auto_metric-cosine.sav", "wb"))